In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
 import nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed, Dropout
from keras import backend as K
from keras import optimizers
from keras.models import Model
import nltk
import re
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
from nltk import tokenize
import seaborn as sns
from sklearn.utils import shuffle

Using TensorFlow backend.


In [4]:
data_frame = shuffle(pd.read_csv('drive/My Drive/Colab Notebooks/dataset/yelp2014train.ss',sep='\t\t', names=["user", "business(product)", "rating", "review"])).reset_index()
data_frame = data_frame[0:60000]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df = data_frame[['review', 'rating']]
rating = df['rating']
review = df['review']

In [6]:
rating.shape

(60000,)

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [0]:
import re
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [0]:
texts = []
for idx in range(df.review.shape[0]):
    text = clean_str(df.review[idx])
    texts.append(text)

# text = rating.apply(lambda x : clean_str(x))

In [0]:
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(texts)
# word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

labels = pd.get_dummies(rating)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,labels, test_size = 0.20)

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, SpatialDropout1D

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('drive/My Drive/Colab Notebooks/weights/LSTM_model.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto') 
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=512, callbacks=[checkpoint])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 563s 12ms/step - loss: 1.3404 - acc: 0.4097 - val_loss: 1.1662 - val_acc: 0.4607
Epoch 2/20
48000/48000 [==============================] - 564s 12ms/step - loss: 1.1125 - acc: 0.4917 - val_loss: 1.1103 - val_acc: 0.4708
Epoch 3/20
48000/48000 [==============================] - 569s 12ms/step - loss: 1.0185 - acc: 0.5365 - val_loss: 1.0740 - val_acc: 0.5078
Epoch 4/20
48000/48000 [==============================] - 564s 12ms/step - loss: 0.9458 - acc: 0.5810 - val_loss: 1.0561 - val_acc: 0.5243
Epoch 5/20
13312/48000 [=======>......................] - ETA: 6:20 - loss: 0.8662 - acc: 0.6146